<a href="https://colab.research.google.com/github/harshitha-gokulraj/Climate-Smart-Crop-Selection-/blob/main/Sustainable_agriculture_jpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SET UP ENVIRONMENT**

In [ ]:
!pip install numpy pandas geopandas rasterio shapely pyproj scikit-learn lightgbm xgboost catboost pyarrow matplotlib plotly streamlit joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 148.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.1 MB/s eta 0:00:00


****

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, label_ranking_average_precision_score
from lightgbm import LGBMClassifier
import joblib


**Dummy Dataset (for testing pipeline)**

In [ ]:
# Create a dummy dataset
np.random.seed(42)
N = 500

data = pd.DataFrame({
    "tmean": np.random.uniform(15, 35, N),       # Mean temperature (°C)
    "rain": np.random.uniform(50, 400, N),       # Rainfall (mm/month)
    "pet": np.random.uniform(100, 250, N),       # PET
    "ph": np.random.uniform(4.5, 8.5, N),        # Soil pH
    "oc": np.random.uniform(0.2, 2.0, N),        # Organic carbon %
    "season": np.random.choice(["kharif","rabi","summer"], N)
})

# Target crops (multi-class for demo)
crops = ["rice","wheat","maize","cotton","soybean"]
data["crop"] = np.random.choice(crops, N)

# Binary suitability (for demo, 70% suitable)
data["suitability"] = np.random.choice([0,1], size=N, p=[0.3,0.7])

data.head()


,tmean,rain,pet,ph,oc,season,crop,suitability
0,22.490802,294.356600,127.769939,6.576327,0.671070,summer,soybean,1
1,34.014286,237.633728,181.285142,6.416728,0.644562,summer,rice,0
2,29.639879,158.334666,230.941875,4.602568,1.831258,summer,maize,1
3,26.973170,334.828257,209.833733,5.864991,0.649183,summer,cotton,1
4,18.120373,289.655910,220.984172,6.020782,0.689510,summer,cotton,0


**Feature Engineering**

In [ ]:
def make_features(df):
    out = df.copy()
    out["rain_cv"] = np.random.uniform(0.1, 0.5, len(df))  # Dummy variability
    out["aridity"] = (out["pet"] / (out["rain"]+1e-6)).clip(0,10)
    out = pd.get_dummies(out, columns=["season"])
    return out

features = make_features(data)
X = features.drop(columns=["crop","suitability"])
y = data["suitability"]


**Model Training**

In [10]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.datasets import load_breast_cancer  # example dataset

# Example dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X))

# Model
clf = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Cross-validation
for tr, va in skf.split(X, y):
    clf.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=[(X.iloc[va], y.iloc[va])],
        eval_metric="average_precision",
        callbacks=[lgb.log_evaluation(0)]  # suppress logs
    )
    oof[va] = clf.predict_proba(X.iloc[va])[:, 1]

# Final score
ap = average_precision_score(y, oof)
print("Average Precision Score:", ap)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 286, number of negative: 169
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4536
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628571 -> initscore=0.526093
[LightGBM] [Info] Start training from score 0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

**Inference – Recommend Crops**

In [13]:
def recommend(sample_features: pd.DataFrame, model):
    score = model.predict_proba(
        sample_features,
        predict_disable_shape_check=True
    )[:, 1]
    return score

# Example input (replace values with real soil & climate data)
sample = pd.DataFrame([{
    "tmean": 27, "rain": 250, "pet": 180, "ph": 6.5, "oc": 1.0,
    "season_kharif": 1, "season_rabi": 0, "season_summer": 0,
    "rain_cv": 0.3, "aridity": 0.72
}])

print("Suitability Score:", recommend(sample, clf))


Suitability Score: [0.99999947]
